In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
pd.set_option('display.max_columns', 150)
pd.set_option('display.max_rows', 150)
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/covid19-global-forecasting-week-2/test.csv
/kaggle/input/covid19-global-forecasting-week-2/train.csv
/kaggle/input/covid19-global-forecasting-week-2/submission.csv


In [2]:
train_filepath = '../input/covid19-global-forecasting-week-2/train.csv'
test_filepath = '../input/covid19-global-forecasting-week-2/test.csv'

In [3]:
train_df = pd.read_csv(train_filepath, index_col="Id")
test_df = pd.read_csv(test_filepath, index_col="ForecastId")

In [4]:
train_df.shape

(20580, 5)

In [5]:
test_df.shape

(12642, 3)

In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20580 entries, 1 to 29370
Data columns (total 5 columns):
Province_State    8750 non-null object
Country_Region    20580 non-null object
Date              20580 non-null object
ConfirmedCases    20580 non-null float64
Fatalities        20580 non-null float64
dtypes: float64(2), object(3)
memory usage: 964.7+ KB


In [7]:
y_train_cc = np.array(train_df['ConfirmedCases'].astype(int))
y_train_ft = np.array(train_df['Fatalities'].astype(int))
cols = ['ConfirmedCases', 'Fatalities']

full_df = pd.concat([train_df.drop(cols, axis=1), test_df])
index_split = train_df.shape[0]
full_df = pd.get_dummies(full_df, columns=full_df.columns)

x_train = full_df[:index_split]
x_test= full_df[index_split:]
#x_train.shape, x_test.shape, y_train_cc.shape, y_train_ft.shape

In [8]:
from sklearn.tree import DecisionTreeRegressor

In [9]:
dt = DecisionTreeRegressor(random_state=1)

In [10]:
dt.fit(x_train,y_train_cc)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=1, splitter='best')

In [11]:
y_pred_cc = dt.predict(x_test)
y_pred_cc = y_pred_cc.astype(int)
y_pred_cc[y_pred_cc <0]=0

In [12]:
dt_f = DecisionTreeRegressor()

dt_f.fit(x_train,y_train_ft)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [13]:
y_pred_ft = dt_f.predict(x_test)
y_pred_ft = y_pred_ft.astype(int)
y_pred_ft[y_pred_ft <0]=0
predicted_df_dt = pd.DataFrame([y_pred_cc, y_pred_ft], index = ['ConfirmedCases','Fatalities'], columns= np.arange(1, y_pred_cc.shape[0] + 1)).T
predicted_df_dt.to_csv('submission_rf.csv', index_label = "ForecastId")

In [14]:
from sklearn.tree import DecisionTreeClassifier

dtcla = DecisionTreeClassifier()
# We train model
dtcla.fit(x_train, y_train_cc)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [15]:
predictions = dtcla.predict(x_test)

In [16]:
dtcla.fit(x_train,y_train_ft)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [17]:
predictions1 = dtcla.predict(x_test)

In [18]:
submission = pd.DataFrame({'ForecastId': test_df.index,'ConfirmedCases':predictions,'Fatalities':predictions1})
filename = 'submission.csv'

submission.to_csv(filename,index=False)

In [19]:
submission

,ForecastId,ConfirmedCases,Fatalities
0,1,22,0
1,2,24,0
2,3,24,0
3,4,40,1
4,5,40,1
...,...,...,...
12637,12638,0,0
12638,12639,0,0
12639,12640,0,0
12640,12641,0,0
